In [2]:
#imports
import pandas as pd
import json
import numpy
import requests


In [3]:
#import biggest cities data
cities_file = 'biggest_cities_per_state.json'
with open(cities_file ,'r') as file:
    city_data = json.loads(file.read())
    
#make into dataframe
cities_info = pd.json_normalize(city_data).T
cities_info.head()

,0
AK,"[[Anchorage, -149.90028, 61.21806], 298695.0]"
AL,"[[Birmingham, -86.80249, 33.52066], 212461.0]"
AR,"[[Little Rock, -92.28959, 34.74648], 197992.0]"
AZ,"[[Phoenix, -112.07404, 33.44838], 1563025.0]"
CA,"[[Los Angeles, -118.24368, 34.05223], 3971883.0]"


In [4]:
#clean df
cities_list = []
lng_list = []
lat_list = []
population_list = []

for i in range(51):
    population_list.append(cities_info[0][i][1])
    cities_list.append(cities_info[0][i][0][0])
    lng_list.append(cities_info[0][i][0][1])
    lat_list.append(cities_info[0][i][0][2])
    
cities_info["City"] = cities_list
cities_info["Latitude"] = lat_list 
cities_info["Longitude"] = lng_list
cities_info["Population"] = population_list
cities_info_clean = cities_info[["City","Latitude","Longitude","Population"]]
cities_info_clean = cities_info_clean.reset_index(drop=False)
cities_info_clean = cities_info_clean.rename(columns={"index":"State"})
cities_info_clean


,State,City,Latitude,Longitude,Population
0,AK,Anchorage,61.21806,-149.90028,298695.0
1,AL,Birmingham,33.52066,-86.80249,212461.0
2,AR,Little Rock,34.74648,-92.28959,197992.0
3,AZ,Phoenix,33.44838,-112.07404,1563025.0
4,CA,Los Angeles,34.05223,-118.24368,3971883.0
5,CO,Denver,39.73915,-104.98470,682545.0
6,CT,Bridgeport,41.17923,-73.18945,147629.0
7,DC,Washington,38.89511,-77.03637,601723.0
8,DE,Wilmington,39.74595,-75.54659,71948.0
9,FL,Jacksonville,30.33218,-81.65565,868031.0


In [15]:
#add data from teleport manually since its not accessable through the api
leisure_culture_score = [3,5,0,7,9,6,0,10,0,6,8,8,6,4,8,7,0,0,7,9,0,3,7,7,7,0,0,6,0,5,0,0,5,6,10,6,5,6,8,4,0,0,7,7,6,0,0,8,7,0,0]
cities_info_clean["Leisure/Culture Score"] = leisure_culture_score
arts_venues = [159,429,0,839,1244,1024,0,926,0,661,1430,495,407,292,1409,878,0,0,541,1279,0,224,1146,968,860,0,0,590,0,685,0,0,398,834,1573,498,630,528,1613,265,0,0,413,955,498,0,0,1059,774,0,0]
#arts venues = art galleries + cinemas + comedy clubs + concert venues + performing arts venues
cities_info_clean["Arts Venues"] = arts_venues
educational_entertainment_venues = [32,95,0,219,621,240,0,579,0,159,456,182,100,63,516,251,0,0,165,498,0,53,417,295,254,0,0,112,0,143,0,0,67,185,636,232,169,177,548,83,0,0,166,392,116,0,0,356,271,0,0]
#ed entertainment = historical sites + museums + zoos
cities_info_clean['Educational Entertainment Venues'] = educational_entertainment_venues
sport_venues = [9,45,0,58,222,133,0,221,0,85,202,25,43,30,214,133,0,0,73,179,0,6,190,165,131,0,0,32,0,64,0,0,10,41,175,82,109,77,212,17,0,0,52,173,15,0,0,132,116,0,0]
cities_info_clean["Sports Venues"] = sport_venues
cities_info_clean['Total Venues'] = cities_info_clean[["Arts Venues",'Educational Entertainment Venues',"Sports Venues"]].sum(axis=1)
lgbt_index = [76.56,67.19,0,70.00,93.19,93.13,0,100.00,0,67.19,70,96.55,93.13,72.81,96.90,72.81,0,0,72.81,96.88,96.88,93.62,63.24,96.88,72.14,0,0,67.58,0,72.81,0,0,93.13,93.13,86.72,66.79,67.67,96.88,80.47,93.13,0,0,70.00,64.44,92.97,0,0,91.25,82.97,0,0]
cities_info_clean['LGBT Equality Index'] = lgbt_index
minorities_tolerance = [.64,.3,0,.4,.44,.62,0,.31,.0,.46,.1,.56,.74,.65,.23,.64,.0,0,.29,.7,.15,.69,.1,.46,.27,.0,.0,.68,0,.42,0,0,.45,.5,.46,.51,.52,.64,.35,.54,0,0,.1,.42,.77,.0,0,.68,.12,0,0]
cities_info_clean["Tolerance of Minorities Score"] = minorities_tolerance
tolerance_score = [7,5,0,6,7,8,0,7,0,6,4,8,8,7,6,7,0,0,5,8,6,8,4,7,5,0,0,7,0,6,0,0,7,7,7,6,6,8,6,7,0,0,4,5,9,0,0,8,5,0,0]
cities_info_clean["Tolerance Score"] = tolerance_score

#drop cities with 0 data


cities_info_clean

,State,City,Latitude,Longitude,Population,Leisure/Culture Score,Arts Venues,Educational Entertainment Venues,Sports Venues,Total Venues,LGBT Equality Index,Tolerance of Minorities Score,Tolerance Score
0,AK,Anchorage,61.21806,-149.90028,298695.0,3,159,32,9,200,76.56,0.64,7
1,AL,Birmingham,33.52066,-86.80249,212461.0,5,429,95,45,569,67.19,0.30,5
2,AR,Little Rock,34.74648,-92.28959,197992.0,0,0,0,0,0,0.00,0.00,0
3,AZ,Phoenix,33.44838,-112.07404,1563025.0,7,839,219,58,1116,70.00,0.40,6
4,CA,Los Angeles,34.05223,-118.24368,3971883.0,9,1244,621,222,2087,93.19,0.44,7
5,CO,Denver,39.73915,-104.98470,682545.0,6,1024,240,133,1397,93.13,0.62,8
6,CT,Bridgeport,41.17923,-73.18945,147629.0,0,0,0,0,0,0.00,0.00,0
7,DC,Washington,38.89511,-77.03637,601723.0,10,926,579,221,1726,100.00,0.31,7
8,DE,Wilmington,39.74595,-75.54659,71948.0,0,0,0,0,0,0.00,0.00,0
9,FL,Jacksonville,30.33218,-81.65565,868031.0,6,661,159,85,905,67.19,0.46,6
